# Assignment 08

## Dimensionality Reduction 

## CSCI E-108

### Steve Elston

> Instructions: For this assignment you will complete the exercises shown. All exercises involve creating and executing some Python code. Additionally, most exercises have questions for you to answer. You can answer questions by creating a Markdown cell and writing your answer. If you are not familiar with Markdown, you can find a brief tutorial here.

## Introduction  

Dimensionality reduction algorithms are widely used in data mining. Human perception of relationships in data is limited beyond a few dimensions. Further, many data mining algorithms produce poor results where when there is significant dependency between the features or variables. In both cases, we can apply dimensionality reduction methods.   

In the exercises in this notebook you will gain some experience working with some commonly used dimensionality reduction methods. Specifically, there are two distinct classes of algorithms you will explore:    
1. **Dimensionality reduction transformation methods** create operators to map a sample (feature) space to an orthogonal space. Typically the original data can be well-represented in lower dimensions in the orthogonal space. Examples of these methods include principle component analysis (PCA) and singular value decomposition (SVD).    
2. **Manifold learning methods** where data in a high dimensional space is mapped onto a lower-dimensional manifold. When the projection is to a low dimension, manifold learning is used to aid visualization of high-dimensional data. Alternatively, manifold learning can be used as a nonlinear map for dimensionality reduction for further analysis.   

To begin, execute the code in the cell below to import the packages you will need. 

In [ ]:
import numpy as np
import pandas as pd
from sklearn.datasets import load_iris
from sklearn.decomposition import PCA, KernelPCA
from sklearn.manifold import SpectralEmbedding, MDS, TSNE
from sklearn.cluster import KMeans, MiniBatchKMeans
from sklearn import random_projection
import umap
from sklearn.metrics import silhouette_score, calinski_harabasz_score
import math
import itertools
import matplotlib.pyplot as plt
import seaborn as sns
import umap

## A Synthetic Example

To make the ideas of dimensionality reduction clear, we start with an extremely simple example. In this example, dimensionality reduction is applied to bivariate Normally distributed data. The code in the cell below does the following:  
1. Generate 500 samples from a bivariate, zero-centered Normal distribution with covariance having a high degree of dependency between the variables:  
$$
cov = 
 \begin{bmatrix}
   1.0 & 0.9\\
   0.9 & 1.0
   \end{bmatrix}
$$
2. Print the empirical covariance matrix of the sample.  
3. Plot the simulated data values.

Execute this code.

In [ ]:
def plot_normal(X):
    X = pd.DataFrame(X, columns=['axis1','axis2'])
    _=sns.jointplot(x='axis1', y='axis2', data=X, xlim=(-4.5,4.5), ylim=(-4.5,4.5))

cov = [[1, 0.9], [0.9, 1]]
np.random.seed(367)
Normal_random = np.random.multivariate_normal([0.0,0.0], cov, size=500)
print('The emperical covariance')
print(np.cov(Normal_random[:,0], Normal_random[:,1]))
plot_normal(Normal_random)

Notice the following aspects of these results:  
1. The empirical covariance matrix is very close in values to the covariance matrix used for the simulation.   
2. The scatter plot shows considerable dependency between the two variables. 
3. The marginal distributions of the two variables appear to be close to Normally distributed. 

Next, the code in the cell below does the following.  
1. A PCA object is instantiated and the data fit.   
2. The PCA model is used to transform or project the original data matrix into the new coordinate system.    
3. The empirical covariance is computed and printed. 
4. **Variance ratio** of the two dimensions of the new space is computed and printed. Here, variance ratio is the variance on each dimension of the space divided by the total variance of the data. 
5. A plot of the projected data values in the new coordinate space is plotted. 

Execute this code. 

In [ ]:
Normal_pca = PCA().fit(Normal_random)
simple_pca = Normal_pca.transform(Normal_random)
print('Covariance of the transformed data')
print(np.cov(simple_pca[:,0], simple_pca[:,1]))
print(f"\nThe variance explained ratio = {Normal_pca.explained_variance_ratio_}")
plot_normal(simple_pca)

The PCA transformation of these data appears to have worked as expected. Notice the following:  
1. The diagonal terms of the covariance matrix are significantly different in value, indicating that the first component (axis) projects the majority of the variance of the data.  
2. The off-diagonal terms of the covariance matrix are effectively 0, indicating there is no dependency between the variables in the new coordinate system. 
3. The observation that most of the variability of the projected data are explained by the first component is confirmed by both the variance ratio values and the scatter plot. 
4. The marginal distributions of the two variables are very close to Normal, but with significantly difference scale or variance. 

## First Running Example  

We will now start working with some simple real-world data. The famous Iris dataset was collected by a botanist named Edgar Anderson around 1935. Subsequently, the dataset became famous in data analysis circles when Ronald A Fisher used it as an example for his seminal 1936 paper on discriminate analysis, one of the first true multivariate statistical methods proposed. By modern standards this data set is small (only 150 samples) and simple (only 4 features), but the simplicity will help in understand the methods at hand.   

The code in the cell below loads the data set and transforms it into a de-meaned Pandas data frame with human readable column and species names. Execute this code. 

In [ ]:
iris_data = load_iris()

## Normalize the data values  
temp = (iris_data['data'] - iris_data['data'].mean(axis=0)) / iris_data['data'].std(axis=0)

## Prepare the data frame 
target_species = {0:'Setosa',1:'Versicolour',2:'Virginica'}
species = [target_species[x] for x in iris_data['target']]
iris = pd.DataFrame(temp, columns=['sepal_length','sepal_width','petal_length','petal_width'])
iris['species'] = species
iris_data = temp
iris

Since there are only 4 features in this dataset a pairs plot will help with understanding the relationships in these data. Execute the code below to display the plot. 

In [ ]:
_=sns.pairplot(iris, hue='species')

Examine this plot array. You can see that values samples for the Setosa species are well separated. However there is some overlap between samples from Versicolour and Virginica. Further, and more importantly, it appears that these is considerable redundancy in these plots. This leads one to suspect that there is a high dependency between these cases.  

We can further investigate the dependency between the variables by computing the covariance matrix. Execute the code in the cell below to compute the covariance matrix of the iris data. 

In [ ]:
iris_convariance = pd.DataFrame(np.cov(np.transpose(iris_data)), index=list(iris.columns)[:-1], columns=list(iris.columns)[:-1])
print('Iris covariance matrix')
print(iris_convariance)
sns.heatmap(iris_convariance, cmap='coolwarm');

Several of the off-diagonal terms of the covariance matrix are far from zero. We can conclude that there is significant dependency between the variables.   

## Compute PCA of the iris data   

The first algorithm you will apply to the iris data is linear PCA.  

> **Exercise 08-1:** Compute the PCA of the iris data and plot the explained variance of the components by the following steps:  
> 1. Instantiate a Scikit-learn PCA model object with [sklearn.decomposition.PCA](https://scikit-learn.org/stable/modules/generated/sklearn.decomposition.PCA.html).  
> 2. Fit the model to `iris_data` numpy array using the `fit` method on the model object.  
> 3. Display a scatterplot of with the `explained_variance_ratio_` attribute of the fitted model vs. the component number.  

In [ ]:
def plot_variance_ratio(pca_model, n_components):   
    _=plt.scatter(range(1, n_components + 1), pca_model.explained_variance_ratio_)    
    _=plt.hlines(0,1,n_components, color='red')
    _=plt.xlabel('Component number')
    _=plt.ylabel('Explain variance ratio')
    _=plt.title('Explained variance ratio vs. component number')

## Put your code below 



## Display the results
n_components = 4
plot_variance_ratio(iris_pca, n_components)

> Examine the plot and answer these questions: 
> 1. Does it appear that much of the variance in the data is explained by the first component and why?  
> 2. Is there any substantial difference in the variance explained between the second and third and fourth components?   

> **Answers:**    
> 1.          
> 2.            

> Recall that the variance of the components from the PCA goes as the square of the singular values. You can gain another view of the relationship between the principle components by executing the code below to plot the singular values. 

In [ ]:
_=plt.scatter(range(1, len(iris_pca.singular_values_) + 1), iris_pca.singular_values_)
_=plt.hlines(0,1,len(iris_pca.singular_values_), color='red')
_=plt.xlabel('Component number')
_=plt.ylabel('Singular value')
_=plt.title('Singular value vs. component number')

> 3. Are these singular values consistent with the variance explained for the components?     
> **End of exercise.**

> **Answer 3:**    

Next, you will investigate the principle components used to project the data into the new space. Execute the code in the cell below to print the components.  

In [ ]:
components = iris_pca.components_
components

The components are in the columns of this array. These components are the projections of the origianl sample space onto the orthogonal space.  

> **Exercise 08-2:** The principle components must be unitary, unit norm, and orthogonal. Do the following to verify these properties.  
> 1. In the first cell below compute and print the Euclidean norm of these of the components using [numpy.linalg.norm](https://numpy.org/doc/stable/reference/generated/numpy.linalg.norm.html).    
> 2. Using [itertools.combinations](https://docs.python.org/3/library/itertools.html) compute the dot (inner) product of each of pairwise combination of the components using [numpy.dot](https://numpy.org/doc/stable/reference/generated/numpy.dot.html) in the second cell below. 

In [ ]:
print('The euclidean norm of the components:')
## Put your code below 






> Examine these results. Are these components orthogonal and unitary and how can you tell?   
> **End of exercise.**

> **Answers:**       

> **Exercise 08-3:** From the initial exploration of the variance explained and singular values it is the case that a few components can explain most of the variance. To project the 4-dimensional data space to a lower dimensional space do the following:   
> 1. Examine the break in the curve for both the explained variance and singular values. This breakpoint determines the number of components you should use for the projection. 
> 2. Instantiate the projected data array using a PCA model object, with the `n_components` arguments and apply the `fit_transform` method on the `iris_data`.   
> 3. Plot the transformation of the data with the `plot_pca` function provided. Make sure you save the returned data frame as `pca_projected`.  

In [ ]:
def plot_pca(X, labels, ax=None):
    pca_projected = pd.DataFrame(X, columns=['Component_1','Component_2'])
    pca_projected['labels'] = labels 
    if ax == None: 
        sns.scatterplot(data=pca_projected, x='Component_1', y='Component_2', hue='labels')
    else:       
        sns.scatterplot(data=pca_projected, x='Component_1', y='Component_2', hue='labels', ax=ax)
    return pca_projected

## Put your code below   


## Plot and save the results 
pca_projected = plot_pca(iris_pca, species)

> Examine the plot you have created. Answer the following questions:  
> 1. How well can these clusters be linearly separated and thereby separating the specie classes and why?  
> 2. Notice the different scales of the two projected variables. Is the range of values of the components consistent with the variance of the components?    


> **Answers:**       
> 1.        
> 2.            

> We can check the independence of the components by computing the covariance. In the cell below create and execute code to . to display the covariance of the projected components. 

In [ ]:
## Put your code below.


> 3. Notice the small values of the off-diagonal components. Do these values indicate the components are orthogonal.     
> **End of Exercise.**

> **Answer 3:**    

## Second Example Dataset

The bowl disease gene dataset has high dimensionality, with over 10,000 features. The question is, can this high dimensional space be projected to a lower dimensional space?   

Execute the code in the cell below to load the data set and prepare it for analysis. 

In [ ]:
gene_data = pd.read_csv('../data/ColonDiseaseGeneData-Cleaned.csv')
labels = gene_data.loc[:,'Disease State']
gene_data = gene_data.drop('Disease State', axis=1)

## Normalize the columns 
gene_data = (gene_data - gene_data.mean(axis=0)) / gene_data.std(axis=0)

## Display the results 
print('Shape of the data array = ' + str(gene_data.shape))
print(gene_data.head())

For the 97 subjects there are gene expression values for over 10,000 genes. The limited number of sanples and extreme high dimensionality markes this a challenging problem.     

To get a feel for these data, execute the code in the cell below to create and display a UMAP embedding of the gene data.  

In [ ]:
np.random.seed(4365)
gene_embedding = umap.UMAP().fit_transform(gene_data)

gene_embedding_df = pd.DataFrame(gene_embedding, columns = ['component1', 'component2'])
gene_embedding_df['disease'] = [0 if label=='Ulcerative Colitis (UC)' else 1 for label in labels]  

fig,ax = plt.subplots(figsize=(6,6))
sns.scatterplot(data=gene_embedding_df, x='component1', y='component2', hue='disease', ax=ax);
ax.set_xlabel('component 1');
ax.set_ylabel('component 2');
ax.set_title('UMAP projection of standardized gene dataset');

On the plot, the disaese state is coded as 1 for Crohn's Disease and 0 for Ulcerative Colitis. By examining the plot is apparent that there is one grouping of patients with Crohn's Disease that is isolated from the others. There is another grouping where patients with both conditions appear mixed. These observations confirm that separating this second group by disease is likely to be quite challenging.  

## PCA with Gene Data    

> **Exercise 08-4:** You will now explore the ability of PCA to reduce the dimensionality of the genetics data. To test this idea do the following:   
> 1. Instantiate a PCA object and apply the `fit` method with the `gene_data` as the argument.  
> 2. Print the cumulative sum of the variance explained by applying the [numpy.cumsum](https://numpy.org/doc/stable/reference/generated/numpy.cumsum.html) function to the `explained_variance_ratio_` attribute of the model object. 
> 2. Plot the first 60 components of the `explained_variance_ratio_` attribute of the model object vs. the component number. 
> 3. Execute your code. 

In [ ]:
## Put your code below 




## Display the explained variance ratio vs. component 
n_components = 60
_=plt.scatter(range(1, n_components + 1), gene_pca.explained_variance_ratio_[:n_components])    
_=plt.hlines(0,1,n_components, color='red')
_=plt.xlabel('Component number')
_=plt.ylabel('Explain variance ratio')
_=plt.title('Explained variance ratio vs. component number')

> Study your plot. Notice that the explained variance ratio decreases rapidly with the component number. Answer the following questions:  
> 1. From the cumulative sums of the variance explained, approximately how much of the total variance can be explained by the first 39 components?  
> 2. Does the decay of the variance explained curve indicate that significant dimensionality reduction is possible for these data?   

> **Answers:**    
> 1.            
> 2.               

> Now you will display and examine a pairwise scatter plot of the first components of the PCA decomposition of the genetics data. Execute the code below with the number of components representing a cumulative 60% of the variance.    

In [ ]:
## Display the components
n_components = 8 
gene_components = ['Component_1','Component_2','Component_3','Component_4','Component_5','Component_6','Component_7','Component_8']
gene_pca_8 = PCA(n_components=n_components).fit(gene_data)
gene_pca_projected = pd.DataFrame(gene_pca_8.transform(gene_data), columns=gene_components)
gene_pca_projected['Disease'] = labels
_=sns.pairplot(gene_pca_projected, hue='Disease')

> Examine the plot. Notice that most of the component values of the two disease types have significant overlap. However, in some cases there are differences in the values on the scatter plots and the marginal density plots. What do these relationship indicate about the ability of these components to separation of the two disease conditions?      
> **End of exercise.**  

> **Answer 3:**           

### K-means with PCA    

Clustering these genetic data is extremely challenging, given the high dimensionality, the small number of genes expressing Crohn's disease and the general complexity. We should therefore have realistic expectations for the results.    

The projection of gene data features onto the orthogonal PCA space, can help with creating a better cluster model. In this case, we will create a k-means cluster model using the first 39 components.     

As a first step, we need to determine how many clusters there should be. To do so, execute the code in the cell below and examine the resulting plots.  

In [ ]:
def cluster_search_kmeans(df, nclusts=(2,21)):
    ## If there are cluster assignments in the data frame remove them. 
    if 'cluster_assignments' in df.columns: df.drop(columns='cluster_assignments', inplace=True)
    WCSS=[]
    BCSS=[]
    silhouette=[]
    CH_index = []
    ## Compute total sum of squares
    x_bar = np.mean(df, axis=0)
    TSS = np.sum(np.sum(np.square(df - x_bar)))
    for n in range(nclusts[0],nclusts[1]+1):   
        temp_model = KMeans(n_clusters=n, n_init=10).fit(df) 
        WCSS.append(temp_model.inertia_)
        BCSS.append(TSS - temp_model.inertia_)
        assignments = temp_model.predict(df)
        silhouette.append(silhouette_score(df, assignments))
        CH_index.append(calinski_harabasz_score(df, assignments))
    _, ax = plt.subplots(2,2, figsize=(8,8))    
    ax = ax.flatten()
    ax[0].plot(range(nclusts[0],nclusts[1]+1),WCSS)   
    ax[0].set_xlabel('Number of clusters')
    ax[0].set_ylabel('WCSS')
    ax[1].plot(range(nclusts[0],nclusts[1]+1),BCSS)   
    ax[1].set_xlabel('Number of clusters')
    ax[1].set_ylabel('BCSS')
    ax[2].plot(range(nclusts[0],nclusts[1]+1),silhouette)   
    ax[2].set_xlabel('Number of clusters')
    ax[2].set_ylabel('Silhouette Score')
    ax[3].plot(range(nclusts[0],nclusts[1]+1),CH_index)   
    ax[3].set_xlabel('Number of clusters')
    ax[3].set_ylabel('Calinski Harabasz Score')
     
    
n_components = 39   
gene_pca_transform = gene_pca.transform(gene_data)
gene_pca_transform = pd.DataFrame(gene_pca_transform[:,:n_components], columns=[str(i) for i in range(n_components)])
    
np.random.seed(9686)
cluster_search_kmeans(gene_pca_transform, nclusts=(2,20)) #, label_column='Disease')    

The question now is, how to interpret these plots. There is no clear 'peak' in the Calinski-Harabasz index, except at 2 clusters. 2 clusters just represnts the grouping of the values already observed, and is not particularly interesting. The peak at 10 clusters in the silhouette coefficient plot is more promising. Additionally, there are small, but noticable, breaks in the curves of the Calinski-Harabasz index, WCSS and BCSS at 10 clusters. We will continue to explore a model with 10 clusters. To creste and evaluate the 10-cluster model, execute the code in the cell below.    

In [ ]:
def plot_cluster_assignments(df):    
    fig,ax = plt.subplots(figsize=(7,6))
    sns.scatterplot(data=df, x='component1', y='component2', 
                    hue='cluster_assignments', style='disease', 
                    markers=['o','^'], palette="Paired", ax=ax) 
    ax.legend(bbox_to_anchor=(1.02, 1), loc='upper left', borderaxespad=0)
    ax.set_xlabel('component 1');
    ax.set_ylabel('component 2');
    ax.set_title('UMAP projection of cluster assignments for HR dataset');

n_clustsers=10 #4
n_components=60   
if 'labels' in gene_pca_transform.columns: gene_pca_transform.drop('labels', axis=1, inplace=True)
gene_embedding_df['cluster_assignments'] = KMeans(n_clusters=n_clustsers, n_init=10).fit_predict(gene_pca_transform)

plot_cluster_assignments(gene_embedding_df)

gene_embedding_df.loc[:,['cluster_assignments','disease']].value_counts().sort_index()

This model shows some interesting relationships. Several clusters have only cases with Corhn's disease, coded 1, and others have only cases of colitis. The other clusters are mixed between the two diseases. Notice that several clusters with Corhn's are in a well separated group away for most cases of colitis. It is likely that considerable domain knowledge is required to go further with the interpretation of this model. 

## Manifold Learning  

Manifold learning seeks to map high-dimensional data onto a low-dimensional linear or nonlinear manifold. There are two reasons to apply manifold learning.      
1. To create a low-dimensional projection, $\le 4$, to aid in visualization of complex data relationships.
2. To reduce dimensionality to improve performance of machine learning models, such as unsupervised clustering models.  

To start our exploration of manifold learning we will map to a two dimensional manifold which can be displayed as a plot showing relationships in complex, high dimensional data. 

The Scikit-Learn package contains a large number of linear and nonlinear [manifold learning algorithms](https://scikit-learn.org/stable/modules/manifold.html). Here we will only investiage two possibilities, spectral manifold learning and the [UMAP algorithm](https://umap-learn.readthedocs.io/en/latest/index.html).   

> **Manifold leaning is not clustering!** It is a common misconception that manifold learning models are clustering models. While it is the case that manifold projections of complex data can show grouping of observations, these groupings must not be confused with clusters. In contrast, clustering algorithms compute specific cluster assignments to the observations.  

### Spectral Manifold Learning 

> **Exercise 08-5:** You will now apply the spectral manifold learning to the iris dataset by these steps:   
> 1. Instantiate a [sklearn.manifold.SpectralEmbedding](https://scikit-learn.org/stable/modules/generated/sklearn.manifold.SpectralEmbedding.html#sklearn.manifold.SpectralEmbedding) object with argument `affinity='rbf'`, radial basis function.
> 2. Use the `fit_transform` method with the iris data as the argument. 
> 3. Display the result using the `plot_pca` function.  

In [ ]:
## Put your code below 



pca_projected=plot_pca(iris_spectral, species)

> Examine this plot. Which aspects (groupings) of the data are well separated?  
> **End of exercise.**

> **Answer:**         

### UMAP      

You have been using the UMAP algorithm for visualization of clusters in a previous assignment. You will now work with some properties of this Algorithm. As a starting point, execute the code in the cell below to display the embedding computed with the UMAP algorithm with default arguments. This plot will give you a baseline to compare affects of hyperparameters.    

In [ ]:
np.random.seed(8273)
iris_UMAP = umap.UMAP().fit_transform(iris_data)
umap_projected=plot_pca(iris_UMAP, species)

There are a number of key hyperparameters that affect the results produced by the UMAP algorithm. Here, we will focus on two, `n_neighbors` and `min_dist`. You can see additional examples of the effects of changing these hyperparamters in the [UMAP documentation](https://umap-learn.readthedocs.io/en/latest/parameters.html). 

The default values of the two parameters of interest here are, n_neighbors=15 and min_dist=0.1. The code in the cell below tests all possible pairs of hyperparameters that are 0.2 and 5.0 times the default values. The embedding for each hyperparameter pairs is then displayed. Execute this code and examine the result.      

In [ ]:
min_dist_list = [0.5, 0.5, 0.02, 0.02]
n_neighbor_list = [3, 20, 3, 20]
_, ax = plt.subplots(2,2, figsize=(10,10))
ax = ax.flatten()
np.random.seed(6655)
for i, (min_dist, n_neighbors) in enumerate(zip(min_dist_list,n_neighbor_list)):
    iris_UMAP = umap.UMAP(min_dist=min_dist, n_neighbors=n_neighbors).fit_transform(iris_data)
    umap_projected=plot_pca(iris_UMAP, species, ax=ax[i])
    ax[i].set_title('min_dist = ' + str(min_dist) + '  n_neihbors = ' + str(n_neighbors))

> **Exercise 08-6:** Examine these plots. Notice how the clusters change in the embedding. Now answer the following questions:     
> 1. How does changing the value of `n_neighbors` change the characteristics of the clusters shown in the embeddings plotted. Why is this behavior expected?          
> 2. How and for which plots does the display of the clusters on the embedding with different values of `min_dist`?  

> **Answers:**       
> 1.              
> 2.           

### UMAP projection of gene data

The UMAP algorithm allows us to **map from a non-Euclidean space to a Euclidean space**. Here we test **mapping from cosine distance to Euclidean distance**. You can now examine the result of changing the hyperparameter values for the embedding of the gene data. As a first step, we execute the code in the cell below to display a baseline projection using cosine distance and the default hyperparameters for `n_neighbors` and `min_dist`.      

In [ ]:
np.random.seed(8833)
gene_UMAP = umap.UMAP(metric='cosine').fit_transform(gene_data)
umap_projected=plot_pca(gene_UMAP, labels)

Next, execute the code in the cell below to display the embeddings for the pairs of values for `n_neighbors` and `min_dist`.

In [ ]:
n_neighbor_list = [10,50,10,50]
_, ax = plt.subplots(2,2, figsize=(10,10))
ax = ax.flatten()
np.random.seed(1010)
for i, (min_dist, n_neighbors) in enumerate(zip(min_dist_list,n_neighbor_list)):
    gene_UMAP = umap.UMAP(min_dist=min_dist, n_neighbors=n_neighbors, metric='cosine').fit_transform(gene_data)
    umap_projected=plot_pca(gene_UMAP, labels, ax=ax[i])
    ax[i].set_title('min_dist = ' + str(min_dist) + '  n_neihbors = ' + str(n_neighbors))

Examine these plots and notice the following:      
1. The small value of `n_neighors` leads to tighter groups. Whereas, the large values of n_neighbors leads to large, poorly separated groups. This represents the fact that the the `n_neighbors` hyperparameter affects how local or wide the search for nearest neighbors is, which affects the properties of the graph used for the UMAP algorithm.      
2. The smaller value of `min_dist` yields tighter groups. This is expected since the distance between samples on the manifold is smaller.     

### UMAP dimensionality reduction for k-means clustering

We will now create a reduced dimensionality manifold projection with UMAP and apply k-means clustering in this Euclidean space. A 20 dimensional space is used for the clustering, which is a reasonable dimensionality for the k-means algorithm.  The code in the cell below does the following:   
1. Compute a 20 component embedding using cosine distance with `n_neighbors=10` and `min_dist=0.05`.
2. Create a data frame from the projection.
3. Call the cluster search function is used over a range of $k=2$ to $k=20$.  
Execute the code in the cell below.     

In [ ]:
np.random.seed(7662)
n_components = 20  
n_neighbors= 10
min_dist = 0.05
umap_transform = umap.UMAP(min_dist=min_dist, n_neighbors=n_neighbors, n_components=n_components, metric='cosine')
gene_umap_transform = umap_transform.fit_transform(gene_data)
gene_umap_transform = pd.DataFrame(gene_umap_transform, columns=[str(i) for i in range(n_components)])
    
np.random.seed(8686)
cluster_search_kmeans(gene_umap_transform, nclusts=(2,20)) #, label_column='Disease')    

As is usually the case, finding a value of k in these charts is ambiguous. In this case the Calinski-Harabasz index has a maximum peak at $k=10$. There is a small corresponding peak in the silhouette score and the WCSS curve is flattened out at this point. Given these considerations we will use $k=10$.   

The code in the cell below computes a $k=10$ k-means model, using the 20 dimension UMAP manifold embedding, and displays the results. Execute this code.  

In [ ]:
np.random.seed(465)
umap_embedding = umap.UMAP(min_dist=min_dist, n_neighbors=n_neighbors, n_components=2, metric='cosine')
gene_embedding_df = umap_embedding.fit_transform(gene_data)

gene_embedding_df = pd.DataFrame(gene_embedding_df, columns = ['component1', 'component2'])
gene_embedding_df['disease'] = [0 if label=='Ulcerative Colitis (UC)' else 1 for label in labels]  


n_clustsers=10
gene_embedding_df['cluster_assignments'] = KMeans(n_clusters=n_clustsers, n_init=10).fit_predict(gene_umap_transform)

plot_cluster_assignments(gene_embedding_df)

gene_embedding_df.loc[:,['cluster_assignments','disease']].value_counts().sort_index()

> **Exercise 8-07:** Examine these results of this model and answer these questions:
> 1. Compare the graphs of Calinski-Harabasz and Silhouette score for the k-means clustering with UMAP using cosine metric to the k-means clustering with PCA dimensionality reduction. In one or a few sentences discuss one or two key differences you can observe between these cases.
> 2. Compared to the k-means clusters with PCA dimensionality reduction to the k-means clusters with cosine UMAP dimensionality reduction. In one or a few sentences are there any noticeable differences in the cluster assignments?    

> **Answers:**
> 1.      
> 2.         

#### Copyright 2021, 2022, 2023, 2024, 2025 Stephen F Elston. All rights reserved.